# PLIP — Protein ligand interaction profiling

See [the tutorial](/Tutorials/protein-ligand-interactions.ipynb).

In [ ]:
import os
from pathlib import Path

import rush
from pdbtools import pdb_fetch

TOKEN = os.getenv("RUSH_TOKEN")
# You might have a custom deployment url, by default the client will use
# https://tengu.qdx.ai
RUSH_URL = os.getenv("RUSH_URL")

# Define our project information
DESCRIPTION = "plip-inference-notebook"
TAGS = ["rush-py", "plip", "notebook"]
WORK_DIR = Path.home() / "qdx" / "plip-inference-rush-py-demo"
os.makedirs(WORK_DIR, exist_ok=True)

# Get our client, for calling modules and using the rush API
client = await rush.build_provider_with_functions(
    access_token=TOKEN, url=RUSH_URL, batch_tags=TAGS
)

# RUSH_TOKEN and RUSH_URL are discovered automatically if present in the env.
# Alternatively, they can be set explicitly:
# TOKEN = os.getenv("RUSH_TOKEN")
# URL = os.getenv("RUSH_URL")
# client = await rush.build_provider_with_functions(access_token=TOKEN, url=URL, batch_tags=TAGS)

# Fetch example PDB
PDB_ID = "1GIH"
FILE_NAME = f"{PDB_ID}.pdb"
FILE_PATH = WORK_DIR / FILE_NAME

complex = list(pdb_fetch.fetch_structure(PDB_ID))

with open(FILE_PATH, "w") as f:
    for line in complex:
        f.write(line)

# Run PLIP
PLIP_RESOURCES = {"storage": 1024_000}

# Get output conformer
output_conformer, output_txt = await client.plip(
    FILE_PATH, resources=PLIP_RESOURCES
)

output_conformer = await output_conformer.get()

# Get output document
output_txt = await output_txt.get()